<a href="https://colab.research.google.com/github/ZicoDiegoRR/my-sdxl-notebook-colab/blob/main/stable_diffusion_xl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


###<font color="black"> » <b><font color="purple">Information </b>✏️📄</font> <font color="black"> «
#####ㅤ
<small>• Text2Img image is saved in Text2Img folder. </small>

<small>• ControlNet-generated image is saved in ControlNet folder. (requires **Canny**, **Depth_Map**, and/or **Open_Pose** to be checked, as well as the direct link to the reference image) </small>

<small>• Inpainting-generated image is saved in Inpainting folder. (requires **Inpainting** to be checked, as well as inputting the image and the mask image)</small>

<small> • You can't combine Inpainting and ControlNet.</small>

<small>• IP-Adapter doesn't change the image name.</small>

<small>• You can load LoRAs from your Google Drive by inputting their path. As of now, only LoRAs are supported. (requires <b>Save_and_Connect_To_GDrive</b> to be checked)</small>

<small>• For ControlNet, leave the image link blank to use the last generated Text2Img image as the reference. Input "inpaint" to use the last generated Inpainting image. And lastly, input "controlnet" to use the last generated ControlNet image. (requires **Canny**, **Depth_Map**, **Inpainting**, and/or **Open_Pose** to be checked) </small>

***

###<font color="black"> » <b><font color="cyan">Guide </b>🚶🏻📋</font> <font color="black"> «
#####ㅤ

<small> **Prompt:** Basically, this one tells the AI what do you want to see in the image. Sometimes, you have to be strict with your words to align the image with your imagination.

<small> **Model (**checkpoint**):** A saved state during an intense training. This is required to generate the image. The type of model you inputted affects the overall style.

<small> **Model Format:** This is pretty self-explanatory. If you want to use .safetensors model, then set it to "Safe Tensors."

<small> **Steps:** It's simply how many iterations the AI will do in order to generate the image. More doesn't always better. You can look for references online.

<small> **Scale (**Guidance Scale**):** This affects how closely related the image with the prompt. High value can be precise, but low value can add extra uniqueness.

<small> **VAE:** Stands for Variational Autoencoder. It basically controls the color of your image.

<small> **Clip Skip:** Lets the AI skip set amount of layers during generation.

<small> **LoRA:** Stands for Low-Rank Adaptation. It holds weight to be "fed" to the AI. In simple terms, LoRA guides the AI to draw specific characters, style, poses, and so much more. You also need to specify the LoRA's scale, similar to **Scale**.

<small> **ControlNet:** Basically a strict instruction based on the inputted image to generate image closely related to the reference.

<small> **Inpainting:** Redrawing an image, but with certain parts of the image changed, just like editing with Photoshop, but AI does the job for you.

<small> **IP-Adapter:** Similar to LoRA, but only follows the inputted image(s). This is stricter than LoRA and sometimes lacks generalization.

<small> **Negative Prompt:** The reverse version of **Prompt**. Instead of telling the AI what do you want, this tells the AI about what do you want to be removed from the image.

#####⬇️For more information, see the cell below⬇️

###<font color="black"> » <b><font color="red">Installing Dependencies </b>💿</font> <font color="black"> «
#####ㅤRun this cell first before creating images!

In [ ]:
#@markdown <b>Run this first to install essential libraries!</b><br>
#@markdown <small><p>Required to use the generator.
from IPython.display import clear_output
print("⚙️ | Downloading libraries...")
!pip install diffusers
!pip install torch torchvision torchaudio
!pip install -U xformers --index-url https://download.pytorch.org/whl/cu121
!pip install transformers
!pip install accelerate
!pip install opencv-python
!pip install peft
!pip install --upgrade huggingface_hub
!pip install compel
!pip install controlnet-aux
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
clear_output()
print("📁 | All essential libraries have been downloaded.")
print("🖌 | You can start generating images now.")

###<font color="black"> » <b><font color="orange">MultiControlNet<font color="black">, <b><font color="magenta"></b>IP-Adapter<font color="black">, and <b><font color="Lime">Inpainting</b> 🔧</font> <font color="black"> «

In [ ]:
import cv2
import re
import numpy as np
from PIL import Image
from compel import Compel, ReturnedEmbeddingsType
from controlnet_aux import OpenposeDetector
from diffusers import ControlNetModel, StableDiffusionXLPipeline, StableDiffusionXLControlNetPipeline, AutoPipelineForInpainting, AutoencoderKL
from diffusers import DPMSolverMultistepScheduler, DPMSolverSinglestepScheduler, KDPM2DiscreteScheduler, KDPM2AncestralDiscreteScheduler, EulerDiscreteScheduler, EulerAncestralDiscreteScheduler, HeunDiscreteScheduler, LMSDiscreteScheduler, DEISMultistepScheduler, UniPCMultistepScheduler, DDIMScheduler, PNDMScheduler
from diffusers.utils import load_image, make_image_grid
from huggingface_hub import login
from transformers import pipeline as pipe
from transformers import CLIPVisionModelWithProjection
from google.colab import drive
import time
import os
import subprocess
import os.path
import torch
import random
import json

# Function to save the data to a json
def save_last(filename, data, type):
    try:
        if os.path.exists(filename):
            with open(filename, 'r') as file:
                existing_data = json.load(file)
        else:
            existing_data = {}

        if type == "[Text-to-Image]":
            existing_data['text2img'] = data
        elif type == "[ControlNet]":
            existing_data['controlnet'] = data
        elif type == "[Inpainting]":
            existing_data['inpaint'] = data
        with open(filename, 'w') as file:
            json.dump(existing_data, file, indent=4)
    except Exception as e:
        print(f"Error occurred: {e}")

# Function to load last-generated image
def load_last(filename, type):
    try:
        with open(filename, 'r') as file:
            data = json.load(file)
            return data.get(type, None)
    except (FileNotFoundError, json.JSONDecodeError):
        return None

# Function to load the saved data from a json
def load_number(filename):
    try:
        with open(filename, 'r') as file:
            data = json.load(file)
            return data['saved']
    except (FileNotFoundError, KeyError):
        return None

# Function to save the data to a json
def save_number(filename, data):
    with open(filename, 'w') as file:
        json.dump({'saved': data}, file)

# Function to convert image into depth map
def get_depth_map(image, depth_estimator):
    image = depth_estimator(image)["depth"]
    image = np.array(image)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    detected_map = torch.from_numpy(image).float() / 255.0
    depth_map = detected_map.permute(2, 0, 1)
    return depth_map

# Only for display in output, nothing crazy
def get_depth_map_display(image, depth_estimator):
    image = depth_estimator(image)["depth"]
    image = np.array(image)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    return image
# Main function to handle the logic
if __name__ == "__main__":
    torch.backends.cudnn.benchmark=True
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:16"
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
    #@markdown <b>Miscellaneous ➡️💽</b>
    Freeze = False #@param {type:"boolean"}
    T4_Mode = False #@param {type:"boolean"}
    Save_and_Connect_To_GDrive = False #@param {type:"boolean"}
    #@markdown <small><b>Freeze:</b> Using the same seed to generate similar images.</small>

    #@markdown <small><b>T4_Mode:</b> Automatically restarts the runtime to unload the GPU's workload to prevent GPU out of memory error. (recommended for T4 GPU)</small>

    # Handling Google Drive and seed
    folder = "/content/gdrive/MyDrive/" if Save_and_Connect_To_GDrive else "/content/"
    filename = os.path.join(folder, "random_number.json")
    if Save_and_Connect_To_GDrive:
        print("📥 | Connecting to Google Drive...")
        drive.mount('/content/gdrive', force_remount=True)
    saved_number = load_number(filename)
    if not Freeze:
        # Generate a new random number if Freeze is False
        random_number = random.randint(1, 1000000000)
        save_number(filename, random_number)
        saved_number = load_number(filename)
    else:
        # Use the saved number if Freeze is True
        if saved_number is not None:
            saved_number = saved_number
        else:
            print("No saved seed found. Generating new one...")
            random_number = random.randint(1, 1000000000)
            save_number(filename, random_number)
            saved_number = load_number(filename)

    # Handling user's input⬇️
    # ——————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
    #@markdown ***
    #@markdown <b>Image Generation Prompt 🖌</b><br>
    #@markdown <small>What do you want to see in the image?</small><br>
    #@markdown <small>Leave everything unchecked and set the IP-Adapter to "None" to generate using Text2Image pipeline.</small>
    Prompt = "" #@param {type:"string"}
    #@markdown <small> Currently supports HuggingFace and CivitAI models, but also works for any websites. For HuggingFace's model, provide only the repository's ID. For CivitAI and others, provide the direct link to the model. When you're getting 401 error from CivitAI's link, add your token below.</small>
    Model = "" #@param {type:"string"}
    Model_Format = "Safe Tensor (.safetensors)" #@param ["Pickle Tensor (.ckpt)", "Safe Tensor (.safetensors)"]
    #@markdown ***
    #@markdown <b>Image Generation Settings ⚙️</b><br>
    Width = 1024 #@param {type:"slider", min: 512, max:1536, step:64}
    Height = 1024 #@param {type:"slider", min:512, max:1536, step:64}
    Scheduler = "Default (defaulting to the model)" #@param ["Default (defaulting to the model)", "DPM++ 2M", "DPM++ 2M Karras", "DPM++ 2M SDE", "DPM++ 2M SDE Karras", "DPM++ SDE", "DPM++ SDE Karras", "DPM2", "DPM2 Karras", "DPM2 a", "DPM2 a Karras", "DDIM", "PNDM", "Euler", "Euler a", "Heun", "LMS", "LMS Karras", "DEISMultistep", "UniPCMultistep"]
    Steps = 20 #@param {type:"number"}
    Scale = 6 #@param {type:"slider", min:1, max:12, step:0.1}
    #@markdown <small> Only accepts one link for Variational Autoencoder. </small>
    VAE_Link = "" #@param {type:"string"}
    #@markdown <small>Not recommended to go beyond 3 or 4.</small>
    #@markdown <small> This will skip how many layers during generation. Setting "1" means skipping the last layer. </small>
    Clip_Skip = 2 #@param {type:"slider", min:0, max:12, step:1}
    #@markdown ***
    #@markdown <b> LoRA</b> 📁🖌️

    #@markdown <small> You can use multiple direct links to the LoRAs using this format: </small>

    #@markdown <font color="blue"><u><small><small> <link/to/file.safetensors>, <link/to/file.bin>, ... </small></small></font></u>

    #@markdown <small> Make sure to put a space (or not) between comma and the next link. </small>
    LoRA_URLs = "" #@param {type:"string"}
    #@markdown <small> Same as the URLs. </small>
    #@markdown <small> The first numbers represent the weight's scale for the first LoRA you just inputted. The logic also applies to every LoRA. </small>
    Weight_Scale = "" #@param {type:"string"}
    #@markdown <small> Passing CivitAI's token is optional, but required if you're getting 401 Unauthorized error. Do not share your CivitAI's API key with anyone else!</small>
    Token = "" #@param {type:"string"}
    #@markdown ***
    #@markdown <b>ControlNet</b> 🖼️🔧

    #@markdown <small> Adjust the thresholds based on your needs. Put an image link to the <b> Canny_Link </b> for reference. </small>
    minimum_canny_threshold = 100 #@param {type:"slider", min:10, max:500, step:5}
    maximum_canny_threshold = 240 #@param {type:"slider", min:100, max:750, step:5}
    Canny_Link = "" #@param {type:"string"}
    Canny = False #@param {type:"boolean"}
    Canny_Strength = 0.7 #@param {type:"slider", min:0.1, max:1, step:0.1}
    #@markdown ————————————————————————
    #@markdown <small> Put an image link to the <b> DepthMap_Link </b> for reference. </small>
    DepthMap_Link = "" #@param {type:"string"}
    Depth_Map = False #@param {type:"boolean"}
    Depth_Strength = 0.7 #@param {type:"slider", min:0.1, max:1, step:0.1}
    #@markdown ————————————————————————
    #@markdown <small> Put an image link to the <b> OpenPose_Link </b> for reference. </small>
    OpenPose_Link = "" #@param {type:"string"}
    Open_Pose = False #@param {type:"boolean"}
    Open_Pose_Strength = 0.8 #@param {type:"slider", min:0.1, max:1, step:0.1}
    #@markdown ***
    #@markdown <b>Inpainting</b> 🖼️🖌️

    #@markdown <small> Set the <b>Inpainting_Image</b> to "pre-generated text2image image" for last generated Text2Image image, "pre-generated controlnet image" for last ControlNet's generated image, and "previous inpainting image" for last inpainted image. Or you can pick image online using its direct link.</small>
    Inpainting_Image = "pre-generated text2image image" #@param ["pre-generated text2image image", "pre-generated controlnet image", "previous inpainting image"] {allow-input:true}
    Mask_Image = "" #@param {type:"string"}
    Inpainting = False #@param {type:"boolean"}
    Inpainting_Strength = 0.9 #@param {type:"slider", min:0.1, max:1, step:0.1}
    #@markdown ***
    #@markdown <b>IP-Adapter</b> 🖼️📝

    #@markdown <small> You can use multiple direct links to the images using this format: </small>

    #@markdown <small><small><small> https://example1.com/.../file.jpg, https://example2.com/.../file.jpg, ... </small></small></small>

    #@markdown <small> Make sure to put a space between comma and the next link. </small>
    IP_Adapter = "None" #@param ["ip-adapter-plus_sdxl_vit-h.bin", "ip-adapter-plus-face_sdxl_vit-h.bin", "ip-adapter_sdxl_vit-h.bin", "None"]
    IP_Image_Link="" #@param {type:"string"}
    IP_Adapter_Strength= 1 #@param {type:"slider", min:0.1, max:1, step:0.1}
    #@markdown ***
    #@markdown <b>Image Generation Negative Prompt ⛔</b><br>
    #@markdown <small>What you <b>don't</b> want to see in the image? (optional)</small><br>
    Negative_Prompt = "" #@param {type:"string"}
    # ——————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————

    # Selecting image
    if Save_and_Connect_To_GDrive:
        base_path = "/content/gdrive/MyDrive"
    else:
        base_path = "/content"
    last_generation_loading = os.path.join(base_path, "last_generation.json")
    if Canny:
        if Canny_Link == "inpaint":
            Canny_link = load_last(last_generation_loading, 'inpaint')
        elif Canny_Link == "controlnet":
            Canny_link = load_last(last_generation_loading, 'controlnet')
        elif not Canny_Link:
            Canny_link = load_last(last_generation_loading, 'text2img')
        else:
            Canny_link = Canny_Link
        if Canny_link is None and not os.path.exists(Canny_link):
            print("No generated image found. Defaulting to Text-to-Image...")
    if Depth_Map:
        if DepthMap_Link == "inpaint":
            Depthmap_Link = load_last(last_generation_loading, 'inpaint')
        elif DepthMap_Link == "controlnet":
            Depthmap_Link = load_last(last_generation_loading, 'controlnet')
        elif not DepthMap_Link:
            Depthmap_Link = load_last(last_generation_loading, 'text2img')
        else:
            Depthmap_Link = DepthMap_Link
        if Depthmap_Link is None and not os.path.exists(Depthmap_Link):
            print("No generated image found. Defaulting to Text-to-Image...")
    if Open_Pose:
        if OpenPose_Link == "inpaint":
            Openpose_Link = load_last(last_generation_loading, 'inpaint')
        elif OpenPose_Link == "controlnet":
            Openpose_Link = load_last(last_generation_loading, 'controlnet')
        elif not OpenPose_Link:
            Openpose_Link = load_last(last_generation_loading, 'text2img')
        else:
            Openpose_Link = OpenPose_Link
        if Openpose_Link is not None and not os.path.exists(Openpose_Link):
            print("No generated image found. Defaulting to Text-to-Image...")
    active_inpaint = False
    if Inpainting:
        if Canny or Depth_Map or Open_Pose:
            raise TypeError("You checked both ControlNet and Inpainting, which will cause incompatibility issues during your run. As of now, there's no alternative way to merge StableDiffusionXLControlNetPipeline and StableDiffusionXLInpaintingPipeline without causing any issues. Perhaps you want to use only one of them?")
        if not Mask_Image:
            raise ValueError("You checked Inpainting while you're leaving Mask_Image empty. Mask_Image is required for Inpainting!")
        if Inpainting_Image == "pre-generated text2image image":
            inpaint_img = load_last(last_generation_loading, 'text2img')
        elif Inpainting_Image == "pre-generated controlnet image":
            inpaint_img = load_last(last_generation_loading, 'controlnet')
        elif Inpainting_Image == "previous inpainting image":
            inpaint_img = load_last(last_generation_loading, 'inpaint')
        else:
            inpaint_image = Inpainting_Image
        if inpaint_img is not None and os.path.exists(inpaint_img):
            inpaint_image = load_image(inpaint_img).resize((1024, 1024))
            mask_image = load_image(Mask_Image).resize((1024, 1024))
            active_inpaint = True
            display(make_image_grid([inpaint_image, mask_image], rows=1, cols=2))
        else:
            print("No generated image found. Defaulting to Text-to-Image...")
    if not IP_Image_Link and IP_Adapter != "None":
        raise ValueError(f"You selected {IP_Adapter}, but left the IP_Image_Link empty. Please change the IP_Adapter to None or add at least one image in IP_Image_Link!")

    # Logic to handle ControlNet and/or MultiControlNets
    controlnets = []
    images = []
    controlnets_scale = []
    if Canny and Canny_link is not None and os.path.exists(Canny_link):
        controlnets.append(ControlNetModel.from_pretrained("diffusers/controlnet-canny-sdxl-1.0", torch_dtype=torch.float16, use_safetensors=True).to("cuda"))
        print("🏞️ | Converting image with Canny Edge Detection...")
        c_img = load_image(Canny_link)
        image_canny = np.array(c_img)
        image_canny = cv2.Canny(image_canny, minimum_canny_threshold, maximum_canny_threshold)
        image_canny = image_canny[:, :, None]
        image_canny = np.concatenate([image_canny, image_canny, image_canny], axis=2)
        canny_image = Image.fromarray(image_canny)
        print("✅ | Canny Edge Detection is complete.")
        time.sleep(1)
        display(make_image_grid([c_img, canny_image.resize((1024, 1024))], rows=1, cols=2))
        images.append(canny_image.resize((1024, 1024)))
        controlnets_scale.append(Canny_Strength)
    if Depth_Map and Depthmap_Link is not None:
        controlnets.append(ControlNetModel.from_pretrained("diffusers/controlnet-depth-sdxl-1.0", torch_dtype=torch.float16, use_safetensors=True).to("cuda"))
        print("🏞️ | Converting image with Depth Map...")
        image_depth = load_image(Depthmap_Link).resize((1024, 1024))
        depth_estimator = pipe("depth-estimation")
        depth_map = get_depth_map(image_depth, depth_estimator).unsqueeze(0).half().to("cpu")
        images.append(depth_map)
        depth_map_display = Image.fromarray(get_depth_map_display(image_depth, depth_estimator))
        print("✅ | Depth Map is complete.")
        controlnets_scale.append(Depth_Strength)
        time.sleep(1)
        display(make_image_grid([image_depth, depth_map_display], rows=1, cols=2))
    if Open_Pose and Openpose_Link is not None:
        openpose = OpenposeDetector.from_pretrained("lllyasviel/ControlNet").to("cpu")
        controlnets.append(ControlNetModel.from_pretrained("thibaud/controlnet-openpose-sdxl-1.0", torch_dtype=torch.float16).to("cuda"))
        print("🏞️ | Converting image with Open Pose...")
        image_openpose = load_image(Openpose_Link)
        openpose_image = openpose(image_openpose)
        images.append(openpose_image.resize((1024, 1024)))
        print("✅ | Open Pose is done.")
        controlnets_scale.append(Open_Pose_Strength)
        display(make_image_grid([image_openpose, openpose_image.resize((1024, 1024))], rows=1, cols=2))

    image_encoder = CLIPVisionModelWithProjection.from_pretrained(
        "h94/IP-Adapter",
        subfolder="models/image_encoder",
        torch_dtype=torch.float16,
    )

    # Logic to handle VAE
    if VAE_Link:
        if not os.path.exists("/content/VAE"):
            os.mkdir("VAE")
        vae_filename = VAE_Link.replace("/", "_").replace(".", "_") + ".safetensors"
        if VAE_Link.startswith("http"):
            if "civitai.com" in VAE_Link:
                if "?" in VAE_Link or "&" in VAE_Link:
                    vae_link = VAE_Link + "&token=" + Token
                else:
                    vae_link = VAE_Link + "token=" + Token
            else:
                vae_link = VAE_Link
            if not os.path.exists(f"/content/VAE/{vae_filename}"):
                !cd /content/VAE; wget -O "$vae_filename" "$vae_link"
            if not os.path.exists("/content/VAE/config.json"):
                !cd /content/VAE; wget -N https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/vae/config.json
        vae_path = f"/content/VAE/{vae_filename}" if VAE_Link.startswith("http") else VAE_Link
        vae = AutoencoderKL.from_single_file(vae_path, torch_dtype=torch.float16, config="/content/VAE/config.json", local_files_only=True)

    # Logic to differentiate if the model is Hugging Face's repository
    if Model.count("/") == 1:
        if not controlnets and not active_inpaint:
            if VAE_Link:
                pipeline = StableDiffusionXLPipeline.from_pretrained(Model, image_encoder=image_encoder, vae=vae, torch_dtype=torch.float16).to("cuda")
            else:
                pipeline = StableDiffusionXLPipeline.from_pretrained(Model, image_encoder=image_encoder, torch_dtype=torch.float16).to("cuda")
        elif active_inpaint and not controlnets:
            if VAE_Link:
                pipeline = AutoPipelineForInpainting.from_pretrained(Model, image_encoder=image_encoder, vae=vae, torch_dtype=torch.float16).to("cuda")
            else:
                pipeline = AutoPipelineForInpainting.from_pretrained(Model, image_encoder=image_encoder, torch_dtype=torch.float16).to("cuda")
        else:
            if VAE_Link:
                pipeline = StableDiffusionXLControlNetPipeline.from_pretrained(Model, image_encoder=image_encoder, controlnet=controlnets, vae=vae, torch_dtype=torch.float16).to("cuda")
            else:
                pipeline = StableDiffusionXLControlNetPipeline.from_pretrained(Model, image_encoder=image_encoder, controlnet=controlnets, torch_dtype=torch.float16).to("cuda")
    else:
        if not os.path.exists("/content/Checkpoint"):
            os.mkdir("Checkpoint")
        if ".ckpt" in Model_Format:
            format = ".ckpt"
        elif ".safetensors" in Model_Format:
            format = ".safetensors"
        checkpoint_name = f"checkpoint_model{format}"
        if Token and "civitai.com" in Model:
            if "?" in Model or "&" in Model:
                checkpoint_link = f"{Model}&token={Token}"
            else:
                checkpoint_link = f"{Model}token={Token}"
        else:
            checkpoint_link = Model
        Model_folder = checkpoint_link.replace("/", "_")
        Model_path = f"/content/Checkpoint/{Model_folder}/{checkpoint_name}"
        Model_path_folder = f"/content/Checkpoint/{Model_folder}"
        if not os.path.exists(Model_path_folder):
            os.mkdir(Model_path_folder)
            !cd "$Model_path_folder"; wget -O "$checkpoint_name" "$checkpoint_link"
        if not controlnets and not active_inpaint:
            if VAE_Link:
                pipeline = StableDiffusionXLPipeline.from_single_file(Model_path, image_encoder=image_encoder, vae=vae, torch_dtype=torch.float16).to("cuda")
            else:
                pipeline = StableDiffusionXLPipeline.from_single_file(Model_path, image_encoder=image_encoder, torch_dtype=torch.float16).to("cuda")
        elif active_inpaint and not controlnets:
            if VAE_Link:
                pipeline = AutoPipelineForInpainting.from_single_file(Model_path, image_encoder=image_encoder, vae=vae, torch_dtype=torch.float16).to("cuda")
            else:
                pipeline = AutoPipelineForInpainting.from_single_file(Model_path, image_encoder=image_encoder, torch_dtype=torch.float16).to("cuda")
        else:
            if VAE_Link:
                pipeline = StableDiffusionXLControlNetPipeline.from_single_file(Model_path, image_encoder=image_encoder, controlnet=controlnets, vae=vae, torch_dtype=torch.float16).to("cuda")
            else:
                pipeline = StableDiffusionXLControlNetPipeline.from_single_file(Model_path, image_encoder=image_encoder, controlnet=controlnets, torch_dtype=torch.float16).to("cuda")
    pipeline.enable_xformers_memory_efficient_attention()
    generator = torch.Generator("cpu").manual_seed(saved_number)
    pipeline.safety_checker = None
    if Scheduler == "DPM++ 2M":
        pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)
    elif Scheduler == "DPM++ 2M Karras":
        pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config, use_karras_sigmas=True)
    elif Scheduler == "DPM++ 2M SDE":
        pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config, algorithm_type="sde-dpmsolver++")
    elif Scheduler == "DPM++ 2M SDE Karras":
        pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config, algorithm_type="sde-dpmsolver++", use_karras_sigmas=True)
    elif Scheduler == "DPM++ SDE":
        pipeline.scheduler = DPMSolverSinglestepScheduler.from_config(pipeline.scheduler.config)
    elif Scheduler == "DPM++ SDE Karras":
        pipeline.scheduler = DPMSolverSinglestepScheduler.from_config(pipeline.scheduler.config)
    elif Scheduler == "DPM2":
        pipeline.scheduler = KDPM2DiscreteScheduler.from_config(pipeline.scheduler.config)
    elif Scheduler == "DPM2 Karras":
        pipeline.scheduler = KDPM2DiscreteScheduler.from_config(pipeline.scheduler.config, use_karras_sigmas=True)
    elif Scheduler == "DPM2 a":
        pipeline.scheduler = KDPM2AncestralDiscreteScheduler.from_config(pipeline.scheduler.config)
    elif Scheduler == "DPM2 a Karras":
        pipeline.scheduler = KDPM2AncestralDiscreteScheduler.from_config(pipeline.scheduler.config, use_karras_sigmas=True)
    elif Scheduler == "Euler":
        pipeline.scheduler = EulerDiscreteScheduler.from_config(pipeline.scheduler.config)
    elif Scheduler == "Euler a":
        pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(pipeline.scheduler.config)
    elif Scheduler == "Heun":
        pipeline.scheduler = HeunDiscreteScheduler.from_config(pipeline.scheduler.config)
    elif Scheduler == "LMS":
        pipeline.scheduler = LMSDiscreteScheduler.from_config(pipeline.scheduler.config)
    elif Scheduler == "LMS Karras":
        pipeline.scheduler = LMSDiscreteScheduler.from_config(pipeline.scheduler.config, use_karras_sigmas=True)
    elif Scheduler == "DEISMultistep":
        pipeline.scheduler = DEISMultistepScheduler.from_config(pipeline.scheduler.config)
    elif Scheduler == "UniPCMultistep":
        pipeline.scheduler = UniPCMultistepScheduler.from_config(pipeline.scheduler.config)
    elif Scheduler == "DDIM":
        pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config, rescale_betas_zero_snr=True, timestep_spacing="trailing")
    elif Scheduler == "PNDM":
        pipeline.scheduler = PNDMScheduler.from_config(pipeline.scheduler.config)
    compel = Compel(tokenizer=[pipeline.tokenizer, pipeline.tokenizer_2], text_encoder=[pipeline.text_encoder, pipeline.text_encoder_2], returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED, requires_pooled=[False, True], truncate_long_prompts=False)
    conditioning, pooled = compel([Prompt, Negative_Prompt])

    # Logic to load LoRA(s)
    if LoRA_URLs:
        lora_list = []
        lora_path = []
        lora_links = re.split(r"\s*,\s*", LoRA_URLs)
        if not os.path.exists("/content/LoRAs"):
            os.mkdir("LoRAs")
        if not Weight_Scale:
            scales_string = ["1"] * len(lora_links)
        elif Weight_Scale and len(re.split(r",| ,", Weight_Scale)) < len(lora_links):
            scales_string = re.split(r",| ,", Weight_Scale)
            for j in range(len(lora_links) - len(scales_string)):
                scales_string.append("1")
        else:
            scales_string = re.split(r"\s*,\s*", Weight_Scale)
        scales = [float(num) for num in scales_string]

        for i, link in enumerate(lora_links, start=1):
            path_file = os.path.join("/content/LoRAs", link.replace("/", "_").replace(".", "_"))
            if not os.path.exists(path_file) and "http" in link:
                os.makedirs(path_file)
            lora_name = link.replace("/", "_").replace(".", "_")
            lora_file_name = f"lora_{lora_name}.safetensors"
            if "civitai.com" in link and Token:
                if "&" in link or "?" in link:
                    civit_link = f"{link}&token={Token}"
                else:
                    civit_link = f"{link}?token={Token}"
                if not os.path.isfile(os.path.join(path_file, lora_file_name)):
                    !cd "$path_file"; wget -O "$lora_file_name" "$civit_link"
                lora_list.append(lora_file_name)
                lora_path.append(path_file)
            elif not link.startswith("/content"):
                if not os.path.isfile(os.path.join(path_file, lora_file_name)):
                    !cd "$path_file"; wget -O "$lora_file_name" "$link"
                lora_list.append(lora_file_name)
                lora_path.append(path_file)
            else:
                if link.startswith("/content/gdrive/MyDrive"):
                    constructed_gdrive_link = link
                else:
                    constructed_gdrive_link = f"/content/gdrive/MyDrive/{link}"
                link_from_gdrive = constructed_gdrive_link.split("/")
                lora_path.append("/".join([word for word in link_from_gdrive if ".safetensors" not in word]))
                lora_list.append(link_from_gdrive[-1])
        lora_weights = [word for word in lora_list if word.endswith(".safetensors")]
        lora_names = [word.replace(".safetensors", "") for word in lora_weights]
        for p in range(len(lora_weights)):
            pipeline.load_lora_weights(f"{lora_path[p]}/{lora_weights[p]}", adapter_name=lora_names[p])
        pipeline.set_adapters(lora_names, adapter_weights=scales)
        print("LoRAs:")
        for lora in lora_weights:
            print(lora)
    torch.cuda.empty_cache()

    # Logic to handle image(s) for IP-Adapter + display
    if IP_Adapter != "None":
        adapter_image = []
        simple_Url = re.split(r"\s*,\s*", IP_Image_Link)
        for link in simple_Url:
            adapter_image.append(load_image(link))
        adapter_display = [element for element in adapter_image]
        if len(adapter_image) % 3 == 0:
            row = len(adapter_image)/3
        else:
            row = int(len(adapter_image)/3) + 1
            for i in range(3*row - len(adapter_image)):
                adapter_display.append(load_image("https://huggingface.co/IDK-ab0ut/BFIDIW9W29NFJSKAOAOXDOKERJ29W/resolve/main/placeholder.png"))
        print("Image(s) for IP-Adapter:")
        display(make_image_grid([element.resize((1024, 1024)) for element in adapter_display], rows=row, cols=3))
        image_embeds = [adapter_image]
        pipeline.load_ip_adapter("h94/IP-Adapter", subfolder="sdxl_models", weight_name=IP_Adapter)
        pipeline.set_ip_adapter_scale(IP_Adapter_Strength)
    torch.cuda.empty_cache()

    # Generate
    if not controlnets and not active_inpaint: # For Text2Img
        image_save = "[Text-to-Image]"
        if IP_Adapter == "None":
            image = pipeline(
                prompt_embeds=conditioning[0:1],
                pooled_prompt_embeds=pooled[0:1],
                negative_prompt_embeds=conditioning[1:2],
                negative_pooled_prompt_embeds=pooled[1:2],
                num_inference_steps=Steps,
                width=Width,
                height=Height,
                guidance_scale=Scale,
                clip_skip=Clip_Skip,
                generator=generator,
            ).images[0]
        else:
            image = pipeline(
                prompt_embeds=conditioning[0:1],
                pooled_prompt_embeds=pooled[0:1],
                negative_prompt_embeds=conditioning[1:2],
                negative_pooled_prompt_embeds=pooled[1:2],
                num_inference_steps=Steps,
                ip_adapter_image=image_embeds,
                width=Width,
                height=Height,
                guidance_scale=Scale,
                clip_skip=Clip_Skip,
                generator=generator,
            ).images[0]
            pipeline.unload_ip_adapter()
    elif active_inpaint and not controlnets: # For Inpainting
        image_save = "[Inpainting]"
        if IP_Adapter == "None":
            image = pipeline(
                prompt_embeds=conditioning[0:1],
                pooled_prompt_embeds=pooled[0:1],
                negative_prompt_embeds=conditioning[1:2],
                negative_pooled_prompt_embeds=pooled[1:2],
                num_inference_steps=Steps,
                width=Width,
                height=Height,
                guidance_scale=Scale,
                clip_skip=Clip_Skip,
                image=inpaint_image,
                mask_image=mask_image,
                generator=generator,
                strength=Inpainting_Strength,
            ).images[0]
        else:
            image = pipeline(
                prompt_embeds=conditioning[0:1],
                pooled_prompt_embeds=pooled[0:1],
                negative_prompt_embeds=conditioning[1:2],
                negative_pooled_prompt_embeds=pooled[1:2],
                num_inference_steps=Steps,
                ip_adapter_image=image_embeds,
                width=Width,
                height=Height,
                guidance_scale=Scale,
                clip_skip=Clip_Skip,
                generator=generator,
                image=inpaint_image,
                mask_image=mask_image,
                strength=Inpainting_Strength,
            ).images[0]
            pipeline.unload_ip_adapter()
    else: # For ControlNet
        image_save = "[ControlNet]"
        if Inpainting: # Deprecated. Will raise an error if both ControlNet and Inpainting collide.
            '''
            if IP_Adapter == "None":
                image = pipeline(
                    prompt_embeds=conditioning[0:1],
                    pooled_prompt_embeds=pooled[0:1],
                    negative_prompt_embeds=conditioning[1:2],
                    negative_pooled_prompt_embeds=pooled[1:2],
                    clip_skip=Clip_Skip,
                    num_inference_steps=Steps,
                    generator=generator,
                    width=Width,
                    height=Height,
                    image=images,
                    controlnet_conditioning_scale=controlnets_scale,
                    guidance_scale=Scale,
                ).images[0]
            else:
                image = pipeline(
                    prompt_embeds=conditioning[0:1],
                    pooled_prompt_embeds=pooled[0:1],
                    negative_prompt_embeds=conditioning[1:2],
                    negative_pooled_prompt_embeds=pooled[1:2],
                    num_inference_steps=Steps,
                    ip_adapter_image=image_embeds,
                    width=Width,
                    height=Height,
                    guidance_scale=Scale,
                    clip_skip=Clip_Skip,
                    generator=generator,
                    image=images,
                    controlnet_conditioning_scale=controlnets_scale,
                ).images[0]
            '''
        else:
            if IP_Adapter == "None":
                image = pipeline(
                    prompt_embeds=conditioning[0:1],
                    pooled_prompt_embeds=pooled[0:1],
                    negative_prompt_embeds=conditioning[1:2],
                    negative_pooled_prompt_embeds=pooled[1:2],
                    clip_skip=Clip_Skip,
                    num_inference_steps=Steps,
                    generator=generator,
                    width=Width,
                    height=Height,
                    image=images,
                    controlnet_conditioning_scale=controlnets_scale,
                    guidance_scale=Scale,
                ).images[0]
            else:
                image = pipeline(
                    prompt_embeds=conditioning[0:1],
                    pooled_prompt_embeds=pooled[0:1],
                    negative_prompt_embeds=conditioning[1:2],
                    negative_pooled_prompt_embeds=pooled[1:2],
                    num_inference_steps=Steps,
                    ip_adapter_image=image_embeds,
                    width=Width,
                    height=Height,
                    guidance_scale=Scale,
                    clip_skip=Clip_Skip,
                    generator=generator,
                    image=images,
                    controlnet_conditioning_scale=controlnets_scale
                ).images[0]

    # Saving the image
    current_time = time.localtime()
    formatted_time = time.strftime("[%H-%M-%S %B %d, %Y]", current_time)
    if Save_and_Connect_To_GDrive:
        if image_save == "[Text-to-Image]":
            image_save_path = "/content/gdrive/MyDrive/Text2Img"
        elif image_save == "[ControlNet]":
            image_save_path = "/content/gdrive/MyDrive/ControlNet"
        else:
            image_save_path = "/content/gdrive/MyDrive/Inpainting"
    else:
        if image_save == "[Text-to-Image]":
            image_save_path = "/content/Text2Img"
        elif image_save == "[ControlNet]":
            image_save_path = "/content/ControlNet"
        else:
            image_save_path = "/content/Inpainting"
    if not os.path.exists(image_save_path):
            os.makedirs(image_save_path)
    split_prompt = re.split("\s*,\s*", Prompt.replace("<", "").replace(">", ""))
    prompt_name = " ".join(split_prompt)
    generated_image_filename = f"{image_save} {formatted_time} {prompt_name}.jpg"
    generated_image_savefile = f"{image_save_path}/{generated_image_filename}"
    image.save(generated_image_savefile)
    print(f"Image is saved at {generated_image_savefile}.")

    # Handling last generated image
    last_generation_json = os.path.join(base_path, "last_generation.json")
    save_last(last_generation_json, generated_image_savefile, image_save)

    # Displaying the image and restarting the runtime if "T4_Mode" is true
    display(image)
    print(f"Seed: {saved_number}")
    if T4_Mode:
        time.sleep(3)
        os.kill(os.getpid(), 9)